## Dependencies 

In [50]:
!pip install pyaudio SpeechRecognition keyboard


[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [51]:
import pyaudio
import speech_recognition as sr
import keyboard

In [52]:
!pip install pydub SpeechRecognition word2number
!apt-get install ffmpeg  # Note: `apt-get` will not work in a standard Jupyter environment. Use a different approach if on Windows or Mac.



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
'apt-get' is not recognized as an internal or external command,
operable program or batch file.


In [53]:
!pip install nltk



[notice] A new release of pip is available: 23.0.1 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [54]:
import re
from word2number import w2n
import nltk
from nltk import word_tokenize, pos_tag, sent_tokenize
import json
from IPython.display import display, Javascript
import base64
import speech_recognition as sr
from pydub import AudioSegment
import io

# Download NLTK resources
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dxl33\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\dxl33\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

## Define Recorder Class

In [32]:
class Recorder:
    def __init__(self, silence_timeout=15):
        self.recognizer = sr.Recognizer()
        self.silence_timeout = silence_timeout  # Time to wait for silence before stopping

    def record_audio(self):
        with sr.Microphone() as source:
            print("Adjusting for ambient noise, please wait...")
            self.recognizer.adjust_for_ambient_noise(source)
            print("Recording... Speak now.")
            
            # Record audio until silence is detected
            audio_data = self.recognizer.listen(source, timeout=self.silence_timeout, phrase_time_limit=self.silence_timeout)
            print("Recording stopped.")
            
            return audio_data


## Define Transcriber Class

In [33]:
class Transcriber:
    def __init__(self):
        self.recognizer = sr.Recognizer()
    
    def transcribe_audio(self, audio_data):
        try:
            print("Transcribing...")
            prompt = self.recognizer.recognize_google(audio_data)
            print("You said: " + prompt)
            return prompt
        except sr.UnknownValueError:
            print("Google Speech Recognition could not understand the audio")
            return ""
        except sr.RequestError as e:
            print(f"Could not request results from Google Speech Recognition service; {e}")
            return ""

## Record Audio

In [40]:
def main():
    # Create a Recorder instance with a silence timeout (e.g., 5 seconds)
    recorder = Recorder(silence_timeout=15)
    transcriber = Transcriber()

    audio_data = recorder.record_audio()
    prompt = transcriber.transcribe_audio(audio_data)
    
    # Check if prompt is empty
    if not prompt:
        print("No transcription available.")
    else:
        print(f"Transcribed Prompt: {prompt}")

    # 'prompt' is now defined and can be used in further processing
    return prompt

# Run the main function and capture the prompt
prompt = main()


Adjusting for ambient noise, please wait...
Recording... Speak now.
Recording stopped.
Transcribing...
You said: I like cheeseburgers you're the best especially with bacon love them
Transcribed Prompt: I like cheeseburgers you're the best especially with bacon love them


In [42]:
#Option to Type Prompt Instead
prompt = "I need to create a table with 3 foot sides and a height of 3 feet and use a leg insert of 4 inches make it 2 inches thick with corner fillets of 4 inches."

In [43]:
speech = prompt

print(speech)

I need to create a table with 3 foot sides and a height of 3 feet and use a leg insert of 4 inches make it 2 inches thick with corner fillets of 4 inches.
